In [1]:
import pandas as pd
import os
import re
import difflib

In [72]:
DATA_PATH = "."
OPENAI_KEY = "/media/hdd2/yusuf/smpl/emse_submission/key_deepseek.txt"
PROMPT_PATH = "prompt.txt"
APPROACH_NAME ="deepseek-chat"

# read files

In [13]:
folder_to_file = {}
for root, _, files in os.walk("."):
    for f in files:
        if f.endswith("sanitized.res.c") and "gpt" not in f and "spinfer" not in f:
            folder = root.split("/")[-1]
            if folder not in folder_to_file:
                folder_to_file[folder] = []

            if f not in folder_to_file[folder]:
                f = f.split(".res.c.sanitized.res.c")[0]
                folder_to_file[folder].append(f)

In [14]:
folder_to_file

{'EXP0-7': ['1533533124_2018-08-06_4fd786e6c3d6_super_btrfs_statfs',
  '1533533124_2018-08-06_4fd786e6c3d6_ctree_add_root_to_dirty_list'],
 'tcf_block_get-61': ['1513791319_2017-12-20_8d1a77f974ca_sch_hfsc_hfsc_change_class'],
 'sock_poll_wait-84': ['1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll'],
 'random_ether_addr-84': ['1529689860_2018-06-22_6c1f0a1ffb7c_ef10_sriov_efx_ef10_sriov_alloc_vf_vswitching'],
 'snd_soc': ['1524629994_2018-04-25_e4b31b816c47_mt8173-afe-pcm_mt8173_afe_i2s_prepare',
  '1524629994_2018-04-25_e4b31b816c47_mtk-afe-fe-dai_mtk_afe_fe_startup'],
 'ttm_bo_init-60': ['1519311151_2018-02-22_724daa4fd65d_nouveau_bo_nouveau_bo_new'],
 'uartlite': ['1360605874_2013-02-11_6d53c3b71d32_uartlite_ulite_isr'],
 'perf_evlist__mmap-69': ['1512266437_2017-12-03_f74b9d3a1ac2_bpf_do_test'],
 'early_memunmap': ['1424769208_2015-02-24_8d4a40bc0651_setup_relocate_initrd'],
 'kees_timer1': ['1508184939_2017-10-16_b9eaf1872222_push-switch_switch_drv_probe',
  '1508184939_2017-10-16

In [15]:
folders = []
files = []
for folder, file_list in folder_to_file.items():
    for file in file_list:
        folders.append(folder)
        files.append(file)

df = pd.DataFrame({
    'folder': folders,
    'file': files
})

In [16]:
df.head(3)

,folder,file
0,EXP0-7,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...
1,EXP0-7,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...
2,tcf_block_get-61,1513791319_2017-12-20_8d1a77f974ca_sch_hfsc_hf...


In [18]:
def open_and_read_file_content(filepath: str)->str:
    """Open the input filepath, then read the content"""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except IOError as e:
        print(f"An error occurred while reading the file: {e}")
    return ""

In [19]:
df["c_before"] = df.apply(lambda x: open_and_read_file_content(os.path.join(DATA_PATH, x.folder, x.file+".c")), axis=1)

In [20]:
df["c_after"] = df.apply(lambda x: open_and_read_file_content(os.path.join(DATA_PATH, x.folder, x.file+".res.c")), axis=1)

# preprocessing

In [21]:
print(df.iloc[0].c_before)

static int btrfs_statfs(struct dentry *dentry, struct kstatfs *buf) {
  struct btrfs_fs_info *fs_info = btrfs_sb(dentry->d_sb);
  struct btrfs_super_block *disk_super = fs_info->super_copy;
  struct list_head *head = &fs_info->space_info;
  struct btrfs_space_info *found;
  u64 total_used = 0;
  u64 total_free_data = 0;
  u64 total_free_meta = 0;
  int bits = dentry->d_sb->s_blocksize_bits;
  __be32 *fsid = (__be32 *)fs_info->fsid;
  unsigned factor = 1;
  struct btrfs_block_rsv *block_rsv = &fs_info->global_block_rsv;
  int ret;
  u64 thresh = 0;
  int mixed = 0;
  rcu_read_lock();
  list_for_each_entry_rcu(found, head, list) {
    if (found->flags & BTRFS_BLOCK_GROUP_DATA) {
      int i;
      total_free_data += found->disk_total - found->disk_used;
      total_free_data -= btrfs_account_ro_block_groups_free_space(found);
      for (i = 0; i < BTRFS_NR_RAID_TYPES; i++) {
        if (!list_empty(&found->block_groups[i])) factor = btrfs_bg_type_to_factor(btrfs_raid_array[i].bg_flag);
 

In [22]:
def remove_comments(c_code:str)->str:
    # Regular expression to match both single-line and multi-line comments
    pattern = r"//.*?$|/\*.*?\*/"
    
    # Substitute the comments with an empty string
    cleaned_code = re.sub(pattern, '', c_code, flags=re.DOTALL | re.MULTILINE)
    
    return cleaned_code.strip()

In [23]:
df["c_before"] = df.c_before.apply(lambda x: remove_comments(x))
df["c_after"] = df.c_after.apply(lambda x: remove_comments(x))

In [24]:
print(df.iloc[0].c_before)

static int btrfs_statfs(struct dentry *dentry, struct kstatfs *buf) {
  struct btrfs_fs_info *fs_info = btrfs_sb(dentry->d_sb);
  struct btrfs_super_block *disk_super = fs_info->super_copy;
  struct list_head *head = &fs_info->space_info;
  struct btrfs_space_info *found;
  u64 total_used = 0;
  u64 total_free_data = 0;
  u64 total_free_meta = 0;
  int bits = dentry->d_sb->s_blocksize_bits;
  __be32 *fsid = (__be32 *)fs_info->fsid;
  unsigned factor = 1;
  struct btrfs_block_rsv *block_rsv = &fs_info->global_block_rsv;
  int ret;
  u64 thresh = 0;
  int mixed = 0;
  rcu_read_lock();
  list_for_each_entry_rcu(found, head, list) {
    if (found->flags & BTRFS_BLOCK_GROUP_DATA) {
      int i;
      total_free_data += found->disk_total - found->disk_used;
      total_free_data -= btrfs_account_ro_block_groups_free_space(found);
      for (i = 0; i < BTRFS_NR_RAID_TYPES; i++) {
        if (!list_empty(&found->block_groups[i])) factor = btrfs_bg_type_to_factor(btrfs_raid_array[i].bg_flag);
 

In [25]:
print(df.iloc[0].c_after)

static int btrfs_statfs(struct dentry *dentry, struct kstatfs *buf) {
  struct btrfs_fs_info *fs_info = btrfs_sb(dentry->d_sb);
  struct btrfs_super_block *disk_super = fs_info->super_copy;
  struct list_head *head = &fs_info->space_info;
  struct btrfs_space_info *found;
  u64 total_used = 0;
  u64 total_free_data = 0;
  u64 total_free_meta = 0;
  int bits = dentry->d_sb->s_blocksize_bits;
  __be32 *fsid = (__be32 *)fs_info->fsid;
  unsigned factor = 1;
  struct btrfs_block_rsv *block_rsv = &fs_info->global_block_rsv;
  int ret;
  u64 thresh = 0;
  int mixed = 0;
  rcu_read_lock();
  list_for_each_entry_rcu(found, head, list) {
    if (found->flags & BTRFS_BLOCK_GROUP_DATA) {
      int i;
      total_free_data += found->disk_total - found->disk_used;
      total_free_data -= btrfs_account_ro_block_groups_free_space(found);
      for (i = 0; i < BTRFS_NR_RAID_TYPES; i++) {
        if (!list_empty(&found->block_groups[i])) factor = btrfs_bg_type_to_factor(btrfs_raid_array[i].bg_flag);
 

In [26]:
def get_loc(input_str: str)->str:
    return len(input_str.splitlines())

In [27]:
df["loc_before"] = df.c_before.apply(lambda x: get_loc(x))
df["loc_after"] = df.c_after.apply(lambda x: get_loc(x))

In [28]:
df.head(3)

,folder,file,c_before,c_after,loc_before,loc_after
0,EXP0-7,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,"static int btrfs_statfs(struct dentry *dentry,...","static int btrfs_statfs(struct dentry *dentry,...",65,65
1,EXP0-7,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,static void add_root_to_dirty_list(struct btrf...,static void add_root_to_dirty_list(struct btrf...,13,13
2,tcf_block_get-61,1513791319_2017-12-20_8d1a77f974ca_sch_hfsc_hf...,static int hfsc_change_class(struct Qdisc *sch...,static int hfsc_change_class(struct Qdisc *sch...,106,106


In [29]:
def get_diff(file_a: str, file_b: str, n_context: int)->str:
    """Get the diff between file_a and file_b using difflib with n_context length"""
    lines_a = file_a.splitlines()
    lines_b = file_b.splitlines()        
    # Generate the unified diff
    diff = difflib.unified_diff(
        lines_a,
        lines_b,
        fromfile="initial",
        tofile="final",
        lineterm='',
        n=n_context
    )

    final_diff = []
    for l in diff:
        if l.startswith("---") or l.startswith("+++"):
            continue
        elif l.startswith("@@ ") and l.endswith(" @@"):
            final_diff.append("<unmodified lines>")
        else:
            final_diff.append(l)

    if final_diff[0] == "<unmodified lines>":
        final_diff = final_diff[1:]

    # Convert the diff generator to a single string
    diff_text = '\n'.join(final_diff)
    return diff_text

In [30]:
df["diff_full"] = df.apply(lambda x: get_diff(x.c_before, x.c_after, 10000000), axis=1)
df["diff_only"] = df.apply(lambda x: get_diff(x.c_before, x.c_after, 0), axis=1)

In [31]:
df["loc_diff_full"] = df.diff_full.apply(lambda x: get_loc(x))
df["loc_diff_only"] = df.diff_only.apply(lambda x: get_loc(x))

In [32]:
df

,folder,file,c_before,c_after,loc_before,loc_after,diff_full,diff_only,loc_diff_full,loc_diff_only
0,EXP0-7,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,"static int btrfs_statfs(struct dentry *dentry,...","static int btrfs_statfs(struct dentry *dentry,...",65,65,static int btrfs_statfs(struct dentry *dentry...,- buf->f_fsid.val[0] ^= BTRFS_I(d_inode(dentr...,67,4
1,EXP0-7,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,static void add_root_to_dirty_list(struct btrf...,static void add_root_to_dirty_list(struct btrf...,13,13,static void add_root_to_dirty_list(struct btr...,- if (root->objectid == BTRFS_EXTENT_TREE_O...,14,2
2,tcf_block_get-61,1513791319_2017-12-20_8d1a77f974ca_sch_hfsc_hf...,static int hfsc_change_class(struct Qdisc *sch...,static int hfsc_change_class(struct Qdisc *sch...,106,106,static int hfsc_change_class(struct Qdisc *sc...,"- err = tcf_block_get(&cl->block, &cl->filter...",107,2
3,sock_poll_wait-84,1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll,"__poll_t tcp_poll(struct file *file, struct so...","__poll_t tcp_poll(struct file *file, struct so...",40,40,"__poll_t tcp_poll(struct file *file, struct s...","- sock_poll_wait(file, wait);\n+ sock_poll_w...",41,2
4,random_ether_addr-84,1529689860_2018-06-22_6c1f0a1ffb7c_ef10_sriov_...,static int efx_ef10_sriov_alloc_vf_vswitching(...,static int efx_ef10_sriov_alloc_vf_vswitching(...,20,20,static int efx_ef10_sriov_alloc_vf_vswitching...,- random_ether_addr(nic_data->vf[i].mac);\n...,21,2
5,snd_soc,1524629994_2018-04-25_e4b31b816c47_mt8173-afe-...,static int mt8173_afe_i2s_prepare(struct snd_p...,static int mt8173_afe_i2s_prepare(struct snd_p...,15,13,static int mt8173_afe_i2s_prepare(struct snd_...,- struct snd_soc_pcm_runtime *rtd = substream...,16,5
6,snd_soc,1524629994_2018-04-25_e4b31b816c47_mtk-afe-fe-...,int mtk_afe_fe_startup(struct snd_pcm_substrea...,int mtk_afe_fe_startup(struct snd_pcm_substrea...,38,37,int mtk_afe_fe_startup(struct snd_pcm_substre...,- struct snd_soc_component *component = snd_s...,39,3
7,ttm_bo_init-60,1519311151_2018-02-22_724daa4fd65d_nouveau_bo_...,"int nouveau_bo_new(struct nouveau_cli *cli, u6...","int nouveau_bo_new(struct nouveau_cli *cli, u6...",73,73,"int nouveau_bo_new(struct nouveau_cli *cli, u...","- ret = ttm_bo_init(&drm->ttm.bdev, &nvbo->bo...",74,2
8,uartlite,1360605874_2013-02-11_6d53c3b71d32_uartlite_ul...,"static irqreturn_t ulite_isr(int irq, void *de...","static irqreturn_t ulite_isr(int irq, void *de...",17,17,"static irqreturn_t ulite_isr(int irq, void *d...",- int stat = ioread32be(port->membase + ULI...,18,2
9,perf_evlist__mmap-69,1512266437_2017-12-03_f74b9d3a1ac2_bpf_do_test,"static int do_test(struct bpf_object *obj, int...","static int do_test(struct bpf_object *obj, int...",72,72,"static int do_test(struct bpf_object *obj, in...","- err = perf_evlist__mmap(evlist, opts.mmap_p...",73,2


# load prompt

In [33]:
prompt_template = open_and_read_file_content(PROMPT_PATH)
print(prompt_template)

# Task
Write a semantic patch for Coccinelle using the SmPL (Semantic Patch Language) based on all diffs specified under **# Input Diffs**. The background knowledge needed to generate semantic patches is specified under **# Background Knowledge**.

# Input Diffs
{input_diffs}

# Output Format
Output the analysis before the final answer. Do not output anything besides the semantic patch in the final answer! Do not include "```" inside the final answer.

## Output Example
[the analysis here]

<final_answer>
[the final semantic patch is here]
</final_answer>

# Background Knowledge
## Collateral Evolutions
The Linux operating system (OS) is evolving rapidly to improve performance and to provide new features. This evolution, however, makes it difficult to maintain platform-specific code such as device drivers. Indeed, an evolution in a driver support library often triggers the need for multiple collateral evolutions in dependent device drivers, to bring the drivers up to date with the new 

In [34]:
df['diff_id'] = df.groupby('folder').cumcount()

In [35]:
df['formatted_diff'] = '<diff_' + df['diff_id'].astype(str) + '>\n' + df['diff_full'] + '\n</diff_' + df['diff_id'].astype(str) + '>\n'

In [36]:
prompt_df = df.groupby('folder').agg({
    'file': list,  # Keep all files as a list
    'formatted_diff': lambda x: '\n'.join(x)  # Concatenate the formatted diffs with newlines
}).reset_index()

In [37]:
prompt_df["formatted_diff"] = prompt_df.formatted_diff.apply(lambda x: prompt_template.format(input_diffs=x))

In [38]:
print(prompt_df.iloc[0].formatted_diff)

# Task
Write a semantic patch for Coccinelle using the SmPL (Semantic Patch Language) based on all diffs specified under **# Input Diffs**. The background knowledge needed to generate semantic patches is specified under **# Background Knowledge**.

# Input Diffs
<diff_0>
 static int btrfs_statfs(struct dentry *dentry, struct kstatfs *buf) {
   struct btrfs_fs_info *fs_info = btrfs_sb(dentry->d_sb);
   struct btrfs_super_block *disk_super = fs_info->super_copy;
   struct list_head *head = &fs_info->space_info;
   struct btrfs_space_info *found;
   u64 total_used = 0;
   u64 total_free_data = 0;
   u64 total_free_meta = 0;
   int bits = dentry->d_sb->s_blocksize_bits;
   __be32 *fsid = (__be32 *)fs_info->fsid;
   unsigned factor = 1;
   struct btrfs_block_rsv *block_rsv = &fs_info->global_block_rsv;
   int ret;
   u64 thresh = 0;
   int mixed = 0;
   rcu_read_lock();
   list_for_each_entry_rcu(found, head, list) {
     if (found->flags & BTRFS_BLOCK_GROUP_DATA) {
       int i;
       tot

In [39]:
prompt_df["loc_formatted_diff"] = prompt_df.formatted_diff.apply(lambda x: get_loc(x))

In [40]:
prompt_df

,folder,file,formatted_diff,loc_formatted_diff
0,EXP0-7,[1533533124_2018-08-06_4fd786e6c3d6_super_btrf...,# Task\nWrite a semantic patch for Coccinelle ...,219
1,dasd_smalloc,[1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_...,# Task\nWrite a semantic patch for Coccinelle ...,295
2,dma_pool_alloc-52,[1518639038_2018-02-15_501017f6d4e1_qla_bsg_ql...,# Task\nWrite a semantic patch for Coccinelle ...,214
3,early_memunmap,[1424769208_2015-02-24_8d4a40bc0651_setup_relo...,# Task\nWrite a semantic patch for Coccinelle ...,171
4,free_bootmem-77,[1540937361_2018-10-30_2013288f7238_init_free_...,# Task\nWrite a semantic patch for Coccinelle ...,148
5,kees_timer1,[1508184939_2017-10-16_b9eaf1872222_push-switc...,# Task\nWrite a semantic patch for Coccinelle ...,310
6,perf_evlist__mmap-69,[1512266437_2017-12-03_f74b9d3a1ac2_bpf_do_test],# Task\nWrite a semantic patch for Coccinelle ...,208
7,random_ether_addr-84,[1529689860_2018-06-22_6c1f0a1ffb7c_ef10_sriov...,# Task\nWrite a semantic patch for Coccinelle ...,156
8,snd_soc,[1524629994_2018-04-25_e4b31b816c47_mt8173-afe...,# Task\nWrite a semantic patch for Coccinelle ...,193
9,sock_poll_wait-84,[1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll],# Task\nWrite a semantic patch for Coccinelle ...,176


# send messages to the model

In [70]:
from openai import OpenAI
from typing import List, Dict
import json

with open(OPENAI_KEY, "r") as f:
    key = f.read()

client = OpenAI(
    api_key=key,
    max_retries=2,
    base_url="https://api.deepseek.com"
)

def extract_final_answer(text: str)->str:
    match = re.search(r"<final_answer>(.*?)</final_answer>", text, re.DOTALL)
    return clean_final_answer(match.group(1)) if match else None

def clean_final_answer(text: str)->str:
    text = text.splitlines()
    cleaned_text = []
    for l in text:
        if l:
            cleaned_text.append(l)
    return "\n".join(cleaned_text)

def format_prompt(prompt: str)->List[Dict]:
    return [
        {"role": "user",
        "content": prompt}
    ]

def send_message_to_model(messages: List[Dict[str, str]],
                          model: str = "deepseek-chat",
                          max_completion_tokens: int = 4096,
                          max_retry: int = 2,
                          ):
    final_response = None
    total_retry = 0
    while final_response == None and total_retry <= max_retry: 
        print(f"Calling API trial {total_retry+1}")
        raw_response = client.chat.completions.create(
            messages=messages,
            model=model,
            max_tokens=max_completion_tokens,
        )
        raw_response = raw_response.to_dict()
        full_answer = raw_response["choices"][0]["message"]["content"]
        final_response = extract_final_answer(full_answer)
        total_retry += 1
    return raw_response, full_answer, final_response

In [62]:
test_message = prompt_df.formatted_diff.iloc[1]
print(test_message)

# Task
Write a semantic patch for Coccinelle using the SmPL (Semantic Patch Language) based on all diffs specified under **# Input Diffs**. The background knowledge needed to generate semantic patches is specified under **# Background Knowledge**.

# Input Diffs
<diff_0>
 static int dasd_eckd_read_conf_lpm(struct dasd_device *device, void **rcd_buffer, int *rcd_buffer_size, __u8 lpm) {
   struct ciw *ciw;
   char *rcd_buf = NULL;
   int ret;
   struct dasd_ccw_req *cqr;
   
   ciw = ccw_device_get_ciw(device->cdev, CIW_TYPE_RCD);
   if (!ciw || ciw->cmd != DASD_ECKD_CCW_RCD) {
     ret = -EOPNOTSUPP;
     goto out_error;
   }
   rcd_buf = kzalloc(DASD_ECKD_RCD_DATA_SIZE, GFP_KERNEL | GFP_DMA);
   if (!rcd_buf) {
     ret = -ENOMEM;
     goto out_error;
   }
-  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 1 , 0, 
-                             device);
+  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 1, 0, device, NULL);
   if (IS_ERR(cqr)) {
     DBF_DEV_EVENT(DBF_WARNING, device, "%s", "C

In [63]:
test_message

'# Task\nWrite a semantic patch for Coccinelle using the SmPL (Semantic Patch Language) based on all diffs specified under **# Input Diffs**. The background knowledge needed to generate semantic patches is specified under **# Background Knowledge**.\n\n# Input Diffs\n<diff_0>\n static int dasd_eckd_read_conf_lpm(struct dasd_device *device, void **rcd_buffer, int *rcd_buffer_size, __u8 lpm) {\n   struct ciw *ciw;\n   char *rcd_buf = NULL;\n   int ret;\n   struct dasd_ccw_req *cqr;\n   \n   ciw = ccw_device_get_ciw(device->cdev, CIW_TYPE_RCD);\n   if (!ciw || ciw->cmd != DASD_ECKD_CCW_RCD) {\n     ret = -EOPNOTSUPP;\n     goto out_error;\n   }\n   rcd_buf = kzalloc(DASD_ECKD_RCD_DATA_SIZE, GFP_KERNEL | GFP_DMA);\n   if (!rcd_buf) {\n     ret = -ENOMEM;\n     goto out_error;\n   }\n-  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 1 , 0, \n-                             device);\n+  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 1, 0, device, NULL);\n   if (IS_ERR(cqr)) {\n     DBF_DEV_EVENT(DBF

In [64]:
input_prompt = format_prompt(test_message)
input_prompt

[{'role': 'user',
  'content': '# Task\nWrite a semantic patch for Coccinelle using the SmPL (Semantic Patch Language) based on all diffs specified under **# Input Diffs**. The background knowledge needed to generate semantic patches is specified under **# Background Knowledge**.\n\n# Input Diffs\n<diff_0>\n static int dasd_eckd_read_conf_lpm(struct dasd_device *device, void **rcd_buffer, int *rcd_buffer_size, __u8 lpm) {\n   struct ciw *ciw;\n   char *rcd_buf = NULL;\n   int ret;\n   struct dasd_ccw_req *cqr;\n   \n   ciw = ccw_device_get_ciw(device->cdev, CIW_TYPE_RCD);\n   if (!ciw || ciw->cmd != DASD_ECKD_CCW_RCD) {\n     ret = -EOPNOTSUPP;\n     goto out_error;\n   }\n   rcd_buf = kzalloc(DASD_ECKD_RCD_DATA_SIZE, GFP_KERNEL | GFP_DMA);\n   if (!rcd_buf) {\n     ret = -ENOMEM;\n     goto out_error;\n   }\n-  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 1 , 0, \n-                             device);\n+  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 1, 0, device, NULL);\n   if (IS_ERR(

In [66]:
raw_response, full_response, final_answer = send_message_to_model(input_prompt)
# final_answer

Calling API trial 1


In [67]:
raw_response

{'id': 'ea6504d6-5bb6-4f01-907c-37ff0c4d6b2a',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '<final_answer>\n@@\nexpression E1, E2, E3, E4;\nidentifier req;\n@@\n\n- dasd_smalloc_request(E1, E2, E3, E4)\n+ dasd_smalloc_request(E1, E2, E3, E4, blk_mq_rq_to_pdu(req))\n  when exists(req);\n\n@@\nexpression E1, E2, E3, E4;\n@@\n\n- dasd_smalloc_request(E1, E2, E3, E4)\n+ dasd_smalloc_request(E1, E2, E3, E4, NULL);\n</final_answer>',
    'role': 'assistant',
    'reasoning_content': 'Okay, I need to create a semantic patch using Coccinelle\'s SmPL based on the given input diffs. Let\'s look at the diffs provided to understand the pattern.\n\nStarting with diff_0:\nThe line changed here is the call to dasd_smalloc_request. The original code had four arguments after the magic number (1, 0, device), and it\'s being changed to add a fifth argument, NULL. So the function call now has an extra NULL at the end.\n\nLooking at diff_1:\nSimilarl

In [59]:
def save_string(output_filepath: str, input_str: str)->None:
    with open(output_filepath, "w") as f:
        f.write(input_str)

def save_json(input_json: Dict, filepath: str)->None:
    with open(filepath, "w") as f:
        json.dump(input_json, f, indent=4)

In [79]:
# def generate_sp_workflow(
#         prompt: str,
#         folder: str,
# ):
#     prompt = format_prompt(prompt)
#     raw_response, full_responses, final_responses = send_message_to_model(prompt)

#     current_folder = os.path.join(DATA_PATH, folder)

#     response_filepath = os.path.join(current_folder, "response.json")
#     save_json(raw_response, response_filepath)

#     for idx, r in enumerate(full_responses):
#         full_response_filepath = os.path.join(current_folder, f"full_response_{idx}.{APPROACH_NAME}.txt")
#         if r:
#             save_string(full_response_filepath, r)
#         else:
#             save_string(full_response_filepath, "")

#     for idx, r in enumerate(final_responses):
#         final_response_filepath = os.path.join(current_folder, f"final_response_{idx}.{APPROACH_NAME}.cocci")
#         if r:
#             save_string(final_response_filepath, r)
#         else:
#             save_string(final_response_filepath, "")

def generate_sp_workflow(
        prompt: str,
        folder: str,
        trial: int = 0,
):
    prompt = format_prompt(prompt)
    raw_response, full_response, final_response = send_message_to_model(prompt)

    current_folder = os.path.join(DATA_PATH, folder)
    
    response_filepath = os.path.join(current_folder, f"response_{trial}.{APPROACH_NAME}.json")
    save_json(raw_response, response_filepath)

    # Save full_response with trial in filename instead of idx
    full_response_filepath = os.path.join(current_folder, f"full_response_{trial}.{APPROACH_NAME}.txt")
    if full_response:
        save_string(full_response_filepath, full_response)
    else:
        save_string(full_response_filepath, "")

    # Save final_response with trial in filename instead of idx
    final_response_filepath = os.path.join(current_folder, f"final_response_{trial}.{APPROACH_NAME}.cocci")
    if final_response:
        save_string(final_response_filepath, final_response)
    else:
        save_string(final_response_filepath, "")

In [80]:
prompt_df

,folder,file,formatted_diff,loc_formatted_diff
0,EXP0-7,[1533533124_2018-08-06_4fd786e6c3d6_super_btrf...,# Task\nWrite a semantic patch for Coccinelle ...,219
1,dasd_smalloc,[1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_...,# Task\nWrite a semantic patch for Coccinelle ...,295
2,dma_pool_alloc-52,[1518639038_2018-02-15_501017f6d4e1_qla_bsg_ql...,# Task\nWrite a semantic patch for Coccinelle ...,214
3,early_memunmap,[1424769208_2015-02-24_8d4a40bc0651_setup_relo...,# Task\nWrite a semantic patch for Coccinelle ...,171
4,free_bootmem-77,[1540937361_2018-10-30_2013288f7238_init_free_...,# Task\nWrite a semantic patch for Coccinelle ...,148
5,kees_timer1,[1508184939_2017-10-16_b9eaf1872222_push-switc...,# Task\nWrite a semantic patch for Coccinelle ...,310
6,perf_evlist__mmap-69,[1512266437_2017-12-03_f74b9d3a1ac2_bpf_do_test],# Task\nWrite a semantic patch for Coccinelle ...,208
7,random_ether_addr-84,[1529689860_2018-06-22_6c1f0a1ffb7c_ef10_sriov...,# Task\nWrite a semantic patch for Coccinelle ...,156
8,snd_soc,[1524629994_2018-04-25_e4b31b816c47_mt8173-afe...,# Task\nWrite a semantic patch for Coccinelle ...,193
9,sock_poll_wait-84,[1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll],# Task\nWrite a semantic patch for Coccinelle ...,176


In [81]:
prompt_df.apply(lambda x: generate_sp_workflow(x.formatted_diff, x.folder, trial=0), axis=1)

Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
dtype: object

In [82]:
prompt_df.apply(lambda x: generate_sp_workflow(x.formatted_diff, x.folder, trial=1), axis=1)

Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
dtype: object

In [83]:
prompt_df.apply(lambda x: generate_sp_workflow(x.formatted_diff, x.folder, trial=2), axis=1)

Calling API trial 1
Calling API trial 2
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1
Calling API trial 1


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
dtype: object